Links
https://op.europa.eu/en/web/eu-vocabularies/e-procurement/tedschemas

https://ec.europa.eu/docsroom/documents/27821/attachments/1/translations/en/renditions/native page 22

#link to files https://1drv.ms/u/s!Ag_DGaic1jWMlY1HKM2GKxf_o76L3w?e=CptwgP

In [ ]:
from pyspark import pandas as ps
import re
import numpy as np
import os
#import pandas as pd

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, concat_ws, lit, col, trim, expr
from pyspark.sql.types import StructType, StructField, StringType,IntegerType

os.environ["PYARROW_IGNORE_TIMEZONE"]="1"

def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster('local[*]')
    conf \
      .set('spark.driver.memory', '64g')\
      .set("fs.s3a.access.key", "minio") \
      .set("fs.s3a.secret.key", "minio123") \
      .set("fs.s3a.endpoint", "http://192.168.1.127:9000") \
      .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
      .set("spark.hadoop.fs.s3a.path.style.access", "true") \
      .set("spark.sql.repl.eagerEval.enabled", "True") \
      .set("spark.sql.adaptive.enabled", "True") \
      .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
      .set("spark.sql.repl.eagerEval.maxNumRows", "10000") \
      .set("spark.setLogLevel", "error")
    
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()

spark = get_spark_session("Falk", SparkConf())

In [ ]:
def xml_to_json(dir_path):
    path = dir_path 
    for filename in os.listdir(path):
        if not filename.endswith('.xml'):
            continue

        fullname = os.path.join(path, filename)

        with open(fullname, 'r') as f:
            xmlString = f.read()
            
        
        jsonString = xmltodict.parse(xmlString, process_namespaces=True, attr_prefix='', cdata_key="text")
        #jsonString = json.dumps(xmltodict.parse(xmlString, process_namespaces=True, attr_prefix='', cdata_key="text"),ensure_ascii=False)
    #jsonString = (xmltodict.parse(xmlString, process_namespaces=True))
    #data = json.load(jsonString)
    #jsondata = json.load(jsonString, object_hook=lambda d: {int(k): [int(i) for i in v] if isinstance(v, list) else v for k, v in d.items()})
    #jsondata2 = json.dumps(jsondata)
    #test.import_bulk(data)
    #test1 = json.load(jsonString, parse_float=float)
    #test.insert(jsonString)
    #print(jsonString)

        #with open(fullname[:-4] + ".json", 'w') as f:
         #   f.write(jsonString)
            
                                     
        with open(fullname[:-4] + ".json", 'w', encoding ='utf8') as json_file:
                                     json.dump(jsonString, json_file, ensure_ascii = False)
                                     

In [ ]:
def _decode(o):
    # Cassting sting to int or float
    
    
    if isinstance(o, str):
        try:
            return int(o)
        except ValueError:
            try:
                return float(o)
            except ValueError:
                return o

    elif isinstance(o, dict):
        return {k: _decode(v) for k, v in o.items()}
    elif isinstance(o, list):
        return [_decode(v) for v in o]
    else:
        return o       
        

In [ ]:
def recursive_iter(obj):
    if isinstance(obj, dict):
        for item in obj.values():
            if "P" in obj and isinstance(obj["P"], list):
                obj["P"] = " ".join([str(e) for e in obj["P"]])
            else:
                yield from recursive_iter(item)
    elif any(isinstance(obj, t) for t in (list, tuple)):
        for item in obj:
            yield from recursive_iter(item)
    else:
        yield obj
        
        

In [ ]:
def json_strip_list2(dir_path):
    path = dir_path 

    for filename in os.listdir(path):
        if not filename.endswith('.json'):
            continue


        fullname = os.path.join(path, filename)
        with open(fullname) as json_file:
            jsonstr = json.load(json_file, object_hook= _decode)
    
        #with open(fullname, 'r') as f:
         #   jsonstr = f.read()

        #json_sting = json.loads(jsonstr)
        for item in recursive_iter(jsonstr):
            with open(fullname[:-4] + "_stript_list.json", 'w', encoding ='utf8') as f:
                json.dump(jsonstr, f, ensure_ascii = False)
            #json_str2 = json.dump(jsonstr)
        
        
        #with open(fullname[:-4] + "_stript_list.json", 'w') as f:
         #   json.dump(json_str2, f)
        
        
        
        #with open(fullname[:-4] + "_stript_list.json", 'w') as f:
         #   f.write(json_str2)   
         
         
       

In [ ]:
from pyspark.sql import *
from copy import copy

# We take a dataframe and return a new one with required changes
def cleanDataFrame(df: DataFrame) -> DataFrame:
    # Returns a new sanitized field name (this function can be anything really)
    def sanitizeFieldName(s: str) -> str:
        return s.replace("-", "_").replace("&", "_").replace("\"", "_")\
            .replace("[", "_").replace("]", "_").replace(".", "_")

    # We call this on all fields to create a copy and to perform any changes we might
    # want to do to the field.
    def sanitizeField(field: StructField) -> StructField:
        field = copy(field)
        field.name = sanitizeFieldName(field.name)
        # We recursively call cleanSchema on all types
        field.dataType = cleanSchema(field.dataType)
        return field

    def cleanSchema(dataType: [DataType]) -> [DateType]:
        dataType = copy(dataType)
        # If the type is a StructType we need to recurse otherwise we can return since
        # we've reached the leaf node
        if isinstance(dataType, StructType):
            # We call our sanitizer for all top level fields
            dataType.fields = [sanitizeField(f) for f in dataType.fields]
        elif isinstance(dataType, ArrayType):
            dataType.elementType = cleanSchema(dataType.elementType)
        return dataType

    # Now since we have the new schema we can create a new DataFrame by using the old Frame's RDD as data and the new schema as the schema for the data
    return spark.createDataFrame(df.rdd, cleanSchema(df.schema))     
    
    
    

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


def flatten_test(df, sep="_"):
    """Returns a flattend dataframe.
        .. versionadded:: x.X.X
        
        Parameters
        ----------
        sep : str
            Delimeter for flatted columns. Default `_`
        
        Notes
        -----
        Don`t use `.` as `sep`
        It won`t work on nested dataframes with more than one level.
        And you will have to use `columns.name`. 
        
        Flattening Maptypes will have to find every key in the column. 
        This can be slow.
        
        Examples
        --------

        data_mixed = [
            {
                "state": "Florida",
                "shortname": "FL",
                "info": {"governor": "Rick Scott"},
                "counties": [
                    {"name": "Dade", "population": 12345},
                    {"name": "Broward", "population": 40000},
                    {"name": "Palm Beach", "population": 60000},
                ],
            },
            {
                "state": "Ohio",
                "shortname": "OH",
                "info": {"governor": "John Kasich"},
                "counties": [
                    {"name": "Summit", "population": 1234},
                    {"name": "Cuyahoga", "population": 1337},
                ],
            },
        ]

        data_mixed = spark.createDataFrame(data=data_mixed)

        data_mixed.printSchema()

        root
        |-- counties: array (nullable = true)
        |    |-- element: map (containsNull = true)
        |    |    |-- key: string
        |    |    |-- value: string (valueContainsNull = true)
        |-- info: map (nullable = true)
        |    |-- key: string
        |    |-- value: string (valueContainsNull = true)
        |-- shortname: string (nullable = true)
        |-- state: string (nullable = true)
        
        
        data_mixed_flat = flatten_test(df, sep=":")
        data_mixed_flat.printSchema()
        root
        |-- shortname: string (nullable = true)
        |-- state: string (nullable = true)
        |-- counties:name: string (nullable = true)
        |-- counties:population: string (nullable = true)
        |-- info:governor: string (nullable = true)
        

        
        
        data = [
            {
                "id": 1,
                "name": "Cole Volk",
                "fitness": {"height": 130, "weight": 60},
            },
            {"name": "Mark Reg", "fitness": {"height": 130, "weight": 60}},
            {
                "id": 2,
                "name": "Faye Raker",
                "fitness": {"height": 130, "weight": 60},
            },
        ]


        df = spark.createDataFrame(data=data)

        df.printSchema()

        root
        |-- fitness: map (nullable = true)
        |    |-- key: string
        |    |-- value: long (valueContainsNull = true)
        |-- id: long (nullable = true)
        |-- name: string (nullable = true)
        
        df_flat = flatten_test(df, sep=":")

        df_flat.printSchema()

        root
        |-- id: long (nullable = true)
        |-- name: string (nullable = true)
        |-- fitness:height: long (nullable = true)
        |-- fitness:weight: long (nullable = true)
        
        data_struct = [
                (("James",None,"Smith"),"OH","M"),
                (("Anna","Rose",""),"NY","F"),
                (("Julia","","Williams"),"OH","F"),
                (("Maria","Anne","Jones"),"NY","M"),
                (("Jen","Mary","Brown"),"NY","M"),
                (("Mike","Mary","Williams"),"OH","M")
                ]

                
        schema = StructType([
            StructField('name', StructType([
                StructField('firstname', StringType(), True),
                StructField('middlename', StringType(), True),
                StructField('lastname', StringType(), True)
                ])),
            StructField('state', StringType(), True),
            StructField('gender', StringType(), True)
            ])

        df_struct = spark.createDataFrame(data = data_struct, schema = schema)

        df_struct.printSchema()

        root
        |-- name: struct (nullable = true)
        |    |-- firstname: string (nullable = true)
        |    |-- middlename: string (nullable = true)
        |    |-- lastname: string (nullable = true)
        |-- state: string (nullable = true)
        |-- gender: string (nullable = true)
        
        df_struct_flat = flatten_test(df_struct, sep=":")

        df_struct_flat.printSchema()

        root
        |-- state: string (nullable = true)
        |-- gender: string (nullable = true)
        |-- name:firstname: string (nullable = true)
        |-- name:middlename: string (nullable = true)
        |-- name:lastname: string (nullable = true)
        """
    # compute Complex Fields (Arrays, Structs and Maptypes) in Schema   
    complex_fields = dict([(field.name, field.dataType)
                            for field in df.schema.fields
                            if type(field.dataType) == ArrayType 
                            or type(field.dataType) == StructType
                            or type(field.dataType) == MapType])
    
    while len(complex_fields) !=0:
        col_name = list(complex_fields.keys())[0]
        #print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))
    
        # if StructType then convert all sub element to columns.
        # i.e. flatten structs
        if (type(complex_fields[col_name]) == StructType):
            expanded = [col(col_name + '.' + k).alias(col_name + sep + k) 
            for k in [n.name for n in complex_fields[col_name]]]
            df = df.select("*", *expanded).drop(col_name)
    
        # if ArrayType then add the Array Elements as Rows using the explode function
        # i.e. explode Arrays
        elif (type(complex_fields[col_name]) == ArrayType):
            df = df.withColumn(col_name, explode_outer(col_name))
        
        # if MapType then convert all sub element to columns.
        # i.e. flatten
        elif (type(complex_fields[col_name]) == MapType):
            keys_df = df.select(explode_outer(map_keys(col(col_name)))).distinct()
            keys = list(map(lambda row: row[0], keys_df.collect()))
            key_cols = list(map(lambda f: col(col_name).getItem(f)
            .alias(str(col_name + sep + f)), keys))
            drop_column_list = [col_name]
            df = df.select([col_name for col_name in df.columns 
            if col_name not in drop_column_list] + key_cols)
    
        # recompute remaining Complex Fields in Schema       
        complex_fields = dict([(field.name, field.dataType)
                            for field in df.schema.fields
                            if type(field.dataType) == ArrayType
                            or type(field.dataType) == StructType
                            or type(field.dataType) == MapType])

    return df

In [ ]:
def json_to_norm(dir_path, path_to_save):
    path = dir_path 

    for filename in os.listdir(path):
        if not filename.endswith('._stript_list.json'):
            continue


        fullname = os.path.join(path, filename)
        with open(fullname) as json_file:
            jsonstr = json.load(json_file)
    
        #with open(fullname, 'r') as f:
         #   jsonstr = f.read()

        #json_sting = json.loads(jsonstr)
        #for item in jsonstr():
        df = spark.read.json(fullname)
        df = cleanDataFrame(df)
        df = flatten(df)
        df.write.mode('append').json(path_to_save)
        #df3.write.mode('append').json('/home/jovyan/notebooks/falk/F02.json')
        #df3.coalesce(1).write.mode('append').json('/home/jovyan/notebooks/falk/F01.json')
        
        
        
      

In [ ]:
#%%
#Then, I define the working directory:

#https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5

working_directory = '/home/jovyan/notebooks/falk/data/new_files_010721'
os.chdir(working_directory)

#After that you can use a combination of the os and zipfile to get where you want:
#run this 4 times! if you get folders zip files in folders ain't unzipt
for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory        